In [1]:
import random 
from numpy import zeros, sqrt, pi, exp,linspace, floor
import pylab 
from statistics import NormalDist
from scipy.optimize import curve_fit
L = 1000
N = 1000
iter = 10000
stats = zeros([2*N, iter], dtype = int)
Nlist = [n for n in range(N)]

NL1 = [i for i in Nlist if i%2 == True]
NL2 = [i for i in Nlist if i%2 == False]
history = zeros([2*N,iter], dtype = int)
NcumIt=[0]
Ncumc = [len(NL1)]

In [2]:
Li = []
for i in range(1,L):
    for j in range(1,L):
        Li.append((i,j))

for n in range(N):
    a = random.choice(Li)
    history[n,0], history[n+N,0] = a[0], a[1]
    Li.remove(a)

In [ ]:

    
    
    


for iterations in range(1,iter):
    space = zeros([L+1,L+1], dtype = int)
    for j in Nlist:
        a = space[history[j,iterations-1], history[j+N, iterations-1]]
        if a>0:
            if j in NL1:
                for i in NL2:
                    if history[j,iterations-1] == history[i,iterations-1] and history[j+N,iterations-1] == history[i+N,iterations-1]:
                        NL1.remove(j)
                        NL2.append(j)
                        break 
                    else:
                        pass 
            if not j in NL1:
                for i in NL1:
                    if history[j,iterations-1] == history[i,iterations-1] and history[j+N,iterations-1] == history[i+N,iterations-1]:
                        NL1.remove(i)
                        NL2.append(i)
                    else:
                        pass 
        if not a>0:
            space[history[j,iterations-1], history[j+N, iterations-1]]+=1

    NcumIt.append(iterations)
    Ncumc.append(len(NL1))
    for n in Nlist:
        dir = random.randint(1,4)
        if dir == 1:
            history[n,iterations], history[n+N,iterations] = history[n,iterations-1]+1, history[n+N,iterations-1]
            if history[n,iterations]>L:
                history[n,iterations]-=2

            

        if dir == 2:
            history[n,iterations], history[n+N,iterations] = history[n,iterations-1]-1, history[n+N,iterations-1]

            if history[n,iterations]<0:
                history[n,iterations]+=2

        if dir == 3:
            history[n,iterations], history[n+N,iterations] = history[n,iterations-1], history[n+N,iterations-1]+1

            if history[n+N,iterations]>L:
                history[n+N,iterations]-=2


        if dir == 4:
            history[n,iterations], history[n+N,iterations] = history[n,iterations-1], history[n+N,iterations-1]-1


            if history[n+N,iterations]<0:
                history[n+N,iterations]+=2

    for n in range(N):
        stats[n,iterations], stats[n+N, iterations] = int(history[n,iterations]-history[n,0]), int(history[n+N,iterations]-history[n+N,0])


In [ ]:
if True==True:
    s=0
    sp=0
    fig = pylab.figure(figsize = (8,8))
    ax = pylab.subplot(111)
    for n in  NL1:
        if s == 0:
            ax.plot(history[n], history[n+N], color = "red", label = "Particle A") 
        else:
            ax.plot(history[n], history[n+N], color = "red") 
        s+=1
    for n in NL2:
        if sp == 0:
            ax.plot(history[n], history[n+N], color = "purple", label = "Particle " + r"$\bar{A}$", alpha = 0.75) 
        else:
            ax.plot(history[n], history[n+N], color = "purple", alpha = 0.75)
        sp+=1
    print(s,sp)
    ax.grid(True, ls = "--", color = "black", alpha = 0.5)
    ax.set_title("Motion of reactive particles in " + r"$\Omega$")
    ax.set_xlim([0,L])
    ax.set_ylim([0,L])
    ax.legend(bbox_to_anchor=(1, 1.05), loc = "center", title = "Particle state")

    
    ax.set_xlabel("x position in " + r"$\Omega$")
    ax.set_ylabel("y position in " + r"$\Omega$")
pylab.show()

In [ ]:
statboi = True
if statboi == True:
    fig = pylab.figure(figsize=(8,8))
    Totx = []
    Toty = []
    ax = pylab.subplot(111)
    for n in NL2:
        for j in range(iter):
            Totx.append(stats[n,j])
            Toty.append(stats[n+N,j])
    sumT = Totx+Toty
    x,bins,p=ax.hist(sumT, bins = (max(sumT) - min(sumT)), density = True, alpha = 0.45, linewidth=3, color = "navy")
    norm = NormalDist.from_samples(sumT)
    mu, sigma = norm.mean, norm.stdev 
    print(mu, sigma)
    sigmap = sigma/sqrt(len(bins))
    print(sigma/sqrt(len(bins)))
    def f(x):
        return 1/(sigma*sqrt(2*pi))*exp(-1/2*((x-mu)/sigma)**2)
    xspace = linspace(min(sumT), max(sumT), 1000)
    ax.plot(xspace, [f(i) for i in xspace], "r--", lw = 3)
    ax.plot([mu,mu], [0,1/(sigma*sqrt(2*pi))], color = "black", lw = 3)
    ax.grid(True, ls= "--", color ="black", alpha = 0.3)
    ax.set_title("Histogram of step deviation from original position for " + r"$\bar{A}$" + " specimen. N = " + str(len(NL2)))
    ax.legend(["Gaussian fit", "Center of Gaussian fit","Histogram data"])
    ax.set_xlabel("Step deviation from starting point")
    ax.set_ylabel("Probability density")
pylab.show()

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D

from scipy.stats import multivariate_normal
from numpy import meshgrid

X = linspace(min(Totx),max(Totx),100)

Y = linspace(min(Toty),max(Toty),100)

X, Y = meshgrid(X,Y)

normx = NormalDist.from_samples(Totx)
mux, sigmax = normx.mean, normx.stdev 

normy = NormalDist.from_samples(Toty)
muy, sigmay = normy.mean, normy.stdev 


pos = zeros(X.shape+(2,))

pos[:,:,0]=X

pos[:,:,1]=Y

rv = multivariate_normal([mux, muy],[[sigmax, 0], [0, sigmay]])

fig = pylab.figure(figsize=(7,7))

ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel("x axis of " + r"$\Omega$")
ax.set_ylabel("y axis of " + r"$\Omega$")
ax.set_zlabel("Probability density")
ax.set_title("Combined Gaussian distribution for each spacial domain")

im = ax.plot_surface(X, Y, rv.pdf(pos), cmap="plasma")
pylab.colorbar(im, shrink = 0.5)
pylab.show()

In [ ]:
if True == True:
    fig = pylab.figure(figsize = (10,10))
    def invexp(t,a,b):
        return a*exp(-b*t)
    
    def invexp2(t,a,b):
        return N - a*exp(-b*t) 

    

    timespace = linspace(0,iterations)
    timeline = NcumIt + [iterations]
    particleC = Ncumc + [Ncumc[-1]]
    particleCp = [N-i for i in particleC]
    prop1, _ = curve_fit(invexp, timeline, particleC)
    alf,bet = prop1 
    print(alf,bet)
    
    prop2,_ = curve_fit(invexp2, timeline, particleCp)
    alf2, bet2 = prop2
    
    

    pylab.plot(timespace, invexp(timespace,alf,bet), color = "navy", lw = 3)
    pylab.plot(timespace, invexp2(timespace,alf2,bet2), color = "lime", lw = 3)
    pylab.plot(timeline,particleC, ls = "--", color = "red", lw = 3)
    pylab.plot(timeline, particleCp, ls = "--", color = "green", lw = 3)
    for i in range(0,len(timeline),50):
        pylab.plot([timeline[i],timeline[i]], [particleC[i], invexp(timeline[i],alf,bet)], "--", color = "black", alpha = 0.3)
        pylab.plot([timeline[i],timeline[i]], [particleCp[i], invexp2(timeline[i],alf2,bet2)], "--", color = "black", alpha = 0.3)

    pylab.legend([ "Exponential fit of form:" + r"$N_0 e^{-bt}$", "Exponential fit of form:" + r"$N - \alpha e^{-\beta t}$", "A particle count", r"$\bar{A}$" + " particle count", "Error"],title = "Legend:" )
    pylab.plot()
    pylab.grid(True, ls= "--", color ="black", alpha = 0.3)
    pylab.title("Particle over iteration count")
    pylab.xlabel("Iteration count [t]")
    pylab.ylabel("Particle count [N]")

pylab.show()